In [ ]:
!pip install wandb -qqq

     |████████████████████████████████| 2.0MB 5.6MB/s 
     |████████████████████████████████| 102kB 8.7MB/s 
     |████████████████████████████████| 133kB 28.0MB/s 
     |████████████████████████████████| 163kB 18.2MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 


In [ ]:
import wandb
import numpy as np
from keras.datasets import fashion_mnist
import tensorflow as tf
from keras.utils import to_categorical
from scipy.special import softmax


In [ ]:
!wandb login

wandb: Currently logged in as: bharatik (use `wandb login --relogin` to force relogin)


In [ ]:
#wandb.init(project="cs6910assignment1", entity="bharatik")

# Question 1: Logging images on wandb

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

def loadData():
  # Load the data (60000 training and 10000 test samples)
  # 10% of training data is for validation 

  fashion_mnist = tf.keras.datasets.fashion_mnist
  (x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
  x_val = x_train[55000:]
  y_val = y_train[55000:]
  x_train = x_train[:55000]
  y_train = y_train[:55000] 
 
  return {
      'x_train': x_train,
      'y_train': y_train, 
      'x_val'  : x_val,
      'y_val'  : y_val,
      'x_test' : x_test,
      'y_test' : y_test,
  }

def display_images():
  dataset = loadData()
  training_images = dataset['x_train']
  training_labels = dataset['y_train']
  image = [];
  label = [];
  for i in range(55000):
    if len(label) >= 10:
      break;
    if class_names[training_labels[i]] not in label:
        image.append(training_images[i])
        label.append(class_names[training_labels[i]])
  wandb.log({"Examples": [ wandb.Image(img, caption=caption) for img, caption in zip(image,label)]})

display_images();

In [ ]:
# Q2: Implementing a feed forward neural network

# Parameters

number_hidden = 2
input_size = 784
output_size = 10
max_iterations = 1000
eta = 0.01

# Preprocessing the data

def preprocess():
  dataset = loadData();
  training_images = dataset['x_train']
  training_labels = dataset['y_train']
  testing_images = dataset['x_test']
  testing_labels = dataset['y_test']

  # Vectorising the images and preprocessing
  train_data = training_images.reshape(training_images.shape[0], input_size)
  test_data = testing_images.reshape(testing_images.shape[0], input_size)
  train_data = train_data/255.0
  test_data  = test_data/255.0

  # One hot encoding for labels
  Enctrain_label = to_categorical(training_labels)
  Enctest_label = to_categorical(testing_labels)
  return train_data, test_data, Enctrain_label, Enctest_label

train_data, test_data, Enctrain_label, Enctest_label = preprocess();

# Creating a network of neurons

def create_network(number_hidden, input_size, output_size):
  #Initialising the parameters
  theta0 = {}
  for i in range(1, number_hidden+2):
    if i <= number_hidden:
      theta0["W" + str(i)] = np.random.randn(input_size,input_size)
      theta0["b" + str(i)] = np.random.randn(input_size,1)
    else:
      theta0["W" + str(i)] = np.random.randn(input_size,output_size)
      theta0["b" + str(i)] = np.random.randn(output_size,1)
  return theta0

# Forward Propogation

def forward_propogation(x, theta, number_hidden, input_size):
  a = {} #pre-activation values
  h = {} #activation values
  h = {"h0": x}

  for i in range(1, number_hidden + 2):
    if i != number_hidden + 1:
      W_current = theta0["W" + str(i)]
      b_current = theta0["b" + str(i)]
      h_previous = h["h" + str(i-1)]
      a_current = np.dot(W_current,h_previous) + b_current
      a["a" + str(i)] = a_current
      h_current = sigmoid_activation(a_current)
      h["h" + str(i)] = h_current

    else:
      a_current = np.dot(W_current,h_previous) + b_current
      h_current = softmax(a_current)  
      h["h" + str(i)] = h_current
      
  y_hat = h["h" + str(number_hidden + 1)]
  return h, a, y_hat


def backward_propogation(h, a, y_hat, Enctrain_label,number_hidden, input_size, output_size):
  return 0;

# This function is used for activation at the hidden layers

def sigmoid_activation(x):
  return 1/(1+np.exp(-x))

# Returns gradient of sigmoid function

def grad_sigmoid(z):
    return (sigmoid_activation(z))*(1 - sigmoid_activation(z))

# This function is used for activation at the output layer

def softmax_activation(x):
  return softmax(x)

# This function calculates the cross entropy loss 

def cross_entropy_loss(p,q):
  p = np.array(p).reshape(-1)
  q = np.array(q).reshape(-1)
  logp = np.log(p)
  loss_vec = (-1)*q*logp
  loss = np.sum(loss_vec)
  return loss    

# Create the gradients

def creategrads( num_hidden, sizes, inputsize, outputsize ):
    sizes = [inputsize] + sizes
    sizes = sizes + [outputsize]
    grads = {"dh0":np.zeros((inputsize,1)),
            "da0":np.zeros((inputsize,1))}
    for i in range(1, num_hidden+2):
        grads["dW" + str(i)] = np.zeros((sizes[i], sizes[i-1]))
        grads["db" + str(i)] = np.zeros((sizes[i],1))
        grads["da" + str(i)] = np.zeros((sizes[i],1))
        grads["dh" + str(i)] = np.zeros((sizes[i],1))
        

    return grads

# Gradient Descent

def gradient_descent(max_iterations, theta, train_data, Enctrain_label, eta, number_hidden, input_size):
  t = 0;
  datapointCount = 0;
  theta0 = create_network(number_hidden, input_size, output_size) # Initialise paramaters
  
  for i in range(max_iterations):

    # Initialise gradients

    for x,y in zip(train_data, Enctrain_label):
        x = train_data[j,:]
        y = Enctest_label[j,:]
        h, a, y_hat = forward_propogation(x, theta, number_hidden, input_size)
        grad_theta = backward_propogation(h, a, y_hat, Enctrain_label, number_hidden, input_size, output_size)

    theta = theta - eta*grad_theta;
    t = t + 1;
  return theta


weights = gradient_descent(max_iterations, theta0, train_data, Enctrain_label, eta, number_hidden, input_size)





In [ ]:
np.arange(1,3)


array([1, 2])